In [ ]:
'''data notes
-when creating nodelist, 421 grants with incorrect year were removed
'''

In [ ]:
import csv
import community
import pandas as pd
import networkx as nx
import math
import matplotlib.pyplot as plt
import pickle
import requests
import sys
import codecs
from bs4 import BeautifulSoup
import re
import os
from datetime import datetime

## Load grantlist and publist

In [ ]:
pickle_in = open("grantcountries.pickle","rb")
grantcountries = pickle.load(pickle_in)
pickle_in = open("totalsoftlist.pickle","rb")
totalsoftlist = pickle.load(pickle_in)
pickle_in = open("totalgrantlist.pickle","rb")
totalgrantlist = pickle.load(pickle_in)
pickle_in = open("totalpublist.pickle","rb")
totalpublist = pickle.load(pickle_in)

In [1]:
#takes date and returns only a year, deals with non-correctly specified dates
def convertDatetoYear(inputdate):
    #datetime_object = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')
    if inputdate:
        try:
            datetime_object = datetime.strptime(inputdate, '%Y-%m-%d')
            return datetime_object.year
        except ValueError:
            if "null" not in inputdate:
                datetime_object = datetime.strptime(inputdate, '%Y-%d-%m')
                return datetime_object.year
            else:
                return ''
    else:
        return ''

In [ ]:
#make nodelist from grants, publications, software entries
def getNodelist(totalgrantlist, totalpublist, totalsoftlist, grantcountries):
    nodelist = list()
    cleangrantlist = list()
    #TODO bring to this format 'type, subtype, id,title,grant,year,countries'
    for grant in totalgrantlist:
        grantyear = convertDatetoYear(grant[1])
        if grant[1]: #if not empty
            nodelist.append(["grant","grant",grant[0],grant[0],grant[0],grantyear, grant[2]])
            cleangrantlist.append(grant[0])
    nodelist1 = len(nodelist)
    for soft in totalsoftlist:
        nodelist.append(["software", "software", soft[0],soft[2],soft[3],soft[1], grantcountries[soft[3]]])
    nodelist2 = len(nodelist) - nodelist1
    for pub in totalpublist:
        idstr = pub[0].replace("\t", "")
        idstr = idstr.replace("\n", "")
        idstr = idstr.strip()
        nodelist.append(["publication", pub[4], idstr, pub[2],pub[3], convertDatetoYear(pub[1]), grantcountries[pub[3]]])
    '''print nodelist[len(nodelist)-1]
    print len(totalgrantlist), len(totalsoftlist), len(totalpublist)
    print nodelist1, nodelist2, len(nodelist) - nodelist1 - nodelist2'''
    return nodelist, cleangrantlist;

In [ ]:
nodelist, cleangrantlist = getNodelist(totalgrantlist, totalpublist, totalsoftlist, grantcountries)
'''print len(nodelist)
print len(cleangrantlist)'''

## Create edgelist from the nodelist

In [ ]:
def getEdgelistFromNodelist(nodelist, cleangrantlist):
    edgelist = list()
    #all the connections are made only through grants
    print len(nodelist)
    for index, node in enumerate(nodelist):
        if node[0] <> 'grant':
            if node[4] in cleangrantlist:
                edgelist.append([node[4], node[2]]) #add edge, where source is grantid and target is recordid
    return edgelist;

In [ ]:
edgelist = getEdgelistFromNodelist(nodelist, cleangrantlist)

## Save nodelist and edgelist

In [ ]:
'''pickle_out = open("edgelist.pickle","wb")
pickle.dump(edgelist, pickle_out)
pickle_out.close()

pickle_out = open("nodelist.pickle","wb")
pickle.dump(nodelist, pickle_out)
pickle_out.close()'''

## Create and save global list of countries from the nodelist

In [ ]:
def getCountryList(nodelist):
    countrieslist = list()
    for node in nodelist:
        if node[0] == 'grant':
            for country in node[6]:
                if country not in countrieslist:
                    countrieslist.append(country)
    return countrieslist;

In [ ]:
countrylist = getCountryList(nodelist)

In [ ]:
'''pickle_out = open("countrylist.pickle","wb")
pickle.dump(countrylist, pickle_out)
pickle_out.close()'''